# Instalación de librerias

Lo primero que debemos hacer es instalar las librerias de Beatiful Soup y requests.

In [1]:
!pip3 install beautifulsoup4 --upgrade
!pip3 install requests
!pip3 install selenium
!pip install webdriver-manager

# Importación de librerias

Vamos a importar las librerias que tenemos que utilizar.

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd
import requests
from os.path  import basename
import time

In [7]:
url = "https://coinmarketcap.com/all/views/all/"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
driver.maximize_window()
driver.get(url)
html = driver.page_source.encode('utf-8')
page_num = 0
time.sleep(5)
try:
    while driver.find_element_by_xpath('//button[contains(text(), "Load More")]'):
        driver.find_element_by_xpath('//button[contains(text(), "Load More")]').click()
        page_num += 1
        print("getting page number "+str(page_num))
        time.sleep(3)
except:
    print("Se ha finalizado la carga de paginas")
    
html = driver.page_source.encode('utf-8')

driver.close() # Cerramos el nevegador.

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [/home/rafaeloga/.wdm/drivers/chromedriver/linux64/94.0.4606.61/chromedriver] found in cache


getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6
getting page number 7
getting page number 8
getting page number 9
getting page number 10
getting page number 11
getting page number 12
getting page number 13
getting page number 14
getting page number 15
getting page number 16
getting page number 17
getting page number 18
getting page number 19
getting page number 20
getting page number 21
getting page number 22
getting page number 23
getting page number 24
getting page number 25
getting page number 26
getting page number 27
getting page number 28
getting page number 29
getting page number 30
getting page number 31
getting page number 32
getting page number 33
getting page number 34
getting page number 35
getting page number 36
getting page number 37
getting page number 38
getting page number 39
getting page number 40
getting page number 41
getting page number 42
getting page number 43
getting page number 

In [8]:
soup = BeautifulSoup(html, "html.parser")
print(soup.title)

<title>All Cryptocurrencies | CoinMarketCap</title>


# Obtención del listado de Cripto Monedas
Lo priemero que vamos a obtener es el listado de las cripto monedas listadas en coinmarketcap. Para ello haremos el analisis de la home donde podemos obtener este listado y algunos destalles de dichas monedas. 

Obtenemos una versión mas facil de trabajar de nuestra pagina web.


In [22]:
print(soup.prettify())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
data=[]
links = soup.find_all('a', class_='cmc-table__column-name--name cmc-link')
links2 = soup.find_all('a', class_='cmc-link')


for row in links:
    link = row['href']
    data.append((link))
    
for link in links2:
    if link['href'].startswith('/currencies/') and not(link['href'].endswith('/markets/')):
        if link['href'] not in data:
            data.append(link['href'])
            
print(len(data))
    
for item in data:
    print(item)


6641
/currencies/bitcoin/
/currencies/ethereum/
/currencies/binance-coin/
/currencies/tether/
/currencies/cardano/
/currencies/xrp/
/currencies/solana/
/currencies/polkadot-new/
/currencies/usd-coin/
/currencies/dogecoin/
/currencies/uniswap/
/currencies/terra-luna/
/currencies/wrapped-bitcoin/
/currencies/binance-usd/
/currencies/litecoin/
/currencies/avalanche/
/currencies/chainlink/
/currencies/bitcoin-cash/
/currencies/shiba-inu/
/currencies/algorand/
/currencies/polygon/
/currencies/stellar/
/currencies/internet-computer/
/currencies/cosmos/
/currencies/vechain/
/currencies/axie-infinity/
/currencies/tron/
/currencies/filecoin/
/currencies/ethereum-classic/
/currencies/ftx-token/
/currencies/bitcoin-bep2/
/currencies/multi-collateral-dai/
/currencies/fantom/
/currencies/theta/
/currencies/hedera/
/currencies/tezos/
/currencies/crypto-com-coin/
/currencies/elrond-egld/
/currencies/pancakeswap/
/currencies/monero/
/currencies/near-protocol/
/currencies/eos/
/currencies/klaytn/
/curr

/currencies/green-shiba-inu-new/
/currencies/wxdai/
/currencies/simbcoin-swap/
/currencies/melo-token/
/currencies/rake-finance/
/currencies/evereth/
/currencies/corgiswap/
/currencies/artery-network/
/currencies/spiderdao/
/currencies/lien/
/currencies/modex/
/currencies/daddyusdt/
/currencies/topchain/
/currencies/nano-dogecoin/
/currencies/alchemy/
/currencies/quantfury-token/
/currencies/petworld/
/currencies/shibaverse/
/currencies/pitbull/
/currencies/mineral/
/currencies/wrapped-ecomi/
/currencies/bitbook/
/currencies/accesslauncher/
/currencies/frmx-token/
/currencies/international-squid-games/
/currencies/jack-token/
/currencies/sportx/
/currencies/defi-warrior/
/currencies/treedefi/
/currencies/pampther/
/currencies/swapz/
/currencies/frenchie-network/
/currencies/taxa-token/
/currencies/rush-coin/
/currencies/sypool/
/currencies/coinxpad/
/currencies/compound-uniswap/
/currencies/gazetv/
/currencies/3x-long-algorand-token/
/currencies/the-luxury-coin/
/currencies/olecoin/
/c

# Scraping de las páginas de cada critpomoneda.
Una vez obtenido el listado de monedas, recorremos cada una de ellas y accedemos a sus precios hitoricos que seran los que almacenemos en nuestro dataset.


In [10]:
from IPython.display import clear_output

dataName=[]
dataSymbol=[]
dataLogo=[]
dataUrl=[]
dataHistoricFile=[]
dataHistoricDate=[]
dataHistoricOpen=[]
dataHistoricHigh=[]
dataHistoricLow=[]
dataHistoricClose=[]
dataHistoricVolume=[]
dataHistoricMarketCap=[]
dataHistoricSymbol=[]
i=0

try:
    driver.close()
except:
    pass
        

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)

#for item in data:
for item in data[:2500]:
    dataHistoricDate=[]
    dataHistoricOpen=[]
    dataHistoricHigh=[]
    dataHistoricLow=[]
    dataHistoricClose=[]
    dataHistoricVolume=[]
    dataHistoricMarketCap=[]
    dataHistoricSymbol=[]
    i=i+1
    print(i)
    try:
        url = "https://coinmarketcap.com" + item +  "historical-data/"
        driver.maximize_window()
        driver.get(url)
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')
        
        
        #Seleccionamos con Selenium el historico de 365 dias
        driver.find_element_by_xpath('//Button[contains(text(), "Date Range")]').click() 
        driver.find_element_by_xpath('//li[contains(text(), "Last 365 days")]').click()
        driver.find_element_by_xpath('//button[contains(text(), "Continue")]').click()
        
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')


        soup = BeautifulSoup(html, "html.parser")

        table = soup.find('table').find('tbody')    
        
        #Parseamos logo y nombre y codigo de la crypto.
        divsTitle = soup.find_all('div', class_='sc-16r8icm-0 gpRPnR nameHeader')
        
        url = item
        dataUrl.append(url)
        
        symbol= divsTitle[0].small.text
        dataSymbol.append(symbol)
    
        name= divsTitle[0].h2.text.replace(symbol,'')
        dataName.append(name)
    
        logoURL = divsTitle[0].img.get('src')
        
        #Parseamos tabla de cotizaciones.
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            dataHistoricSymbol.append(symbol)
            dataHistoricDate.append(tds[0].text)
            dataHistoricOpen.append(tds[1].text)
            dataHistoricHigh.append(tds[2].text)
            dataHistoricLow.append(tds[3].text)
            dataHistoricClose.append(tds[4].text)
            dataHistoricVolume.append(tds[5].text)
            dataHistoricMarketCap.append(tds[6].text)
        
        dictHistoric={'symbol': dataHistoricSymbol, 'date':dataHistoricDate, 'open': dataHistoricOpen,
             'high': dataHistoricHigh, 'low':dataHistoricLow, 'close':dataHistoricClose, 'volume':dataHistoricVolume,
             'marketCap':dataHistoricMarketCap}
        df_hist = pd.DataFrame(dictHistoric)

        df_hist.to_csv("./historicData/"+symbol+".csv")
        
        with open("./images/"+symbol+".png","wb") as f:
                f.write(requests.get(logoURL).content)
        dataLogo.append(symbol+".png")
    
        dataHistoricFile.append(symbol+'.csv')
    
        
        clear_output(wait=True)
        
        dictData={'name': dataName, 'symbol': dataSymbol, 'logo': dataLogo, 'historicalFile': dataHistoricFile, 'url':dataUrl}

        df = pd.DataFrame(dictData)
        
        df.to_csv("coin_info.csv")
       
    except:
        pass
     
driver.close()


2500
